In [ ]:
import os, glob

image_paths = []

for folder in os.listdir("test_images"):
    fileList = glob.glob(f'test_images/{folder}/*')
    image_paths.extend(fileList)

In [ ]:
import random
import pandas as pd

length = len(image_paths)
predictions = [['Cat', 'Dog', 'Wildlife'][random.randint(0,2)] for i in range(length)]

data = {
    'image_id': [i for i in range(length)],
    'pred': predictions ,
    'user_vote_0': [None for i in range(length)],
    'user_vote_1': [None for i in range(length)]
}

pd.DataFrame(data).set_index('image_id').to_csv("test_data.csv")

In [ ]:
import subprocess

def add_a_vote(col, image_id, label):
    df = pd.read_csv("test_data.csv", index_col = "image_id")
    df.at[image_id,col] = label
    df.to_csv("test_data.csv")
    subprocess.run(["git", "pull"])
    subprocess.run(['git', 'add', "test_data.csv"], check=True)
    subprocess.run(['git', 'commit', '-m', "update vote"], check=True)
    subprocess.run(["git","push"])

In [ ]:
import os
from PIL import Image
from IPython.display import display
import ipywidgets as widgets

# Function to display the next image
def show_next_image(images, predictions, idx, img_widget, output_widget):
    
    if idx >= len(images):
        print("No more images to label.")
        return
    
    img_path = images[idx]
    img = Image.open(img_path)
    img = img.resize((256, 256))  # Resize image to 256x256
    
    img_widget.value = img_to_bytes(img)  # Update the image in the widget
    output_widget.clear_output()  # Clear previous output
    first_click = True  # Flag to track if the button was clicked for the first time
    
    def on_button_click(label):
        nonlocal first_click, idx  # Declare as nonlocal to modify within this function
        random_int = random.randint(0,1)
        if first_click and random_int:
            with output_widget:
                display(widgets.HTML(f"""
                    <div style="border: 1px solid #d3d3d3; padding: 10px; border-radius: 5px; background-color: #f9f9f9;">
                        <strong>The model has made the following prediction:</strong> 
                        <span style="color: #0073e6;">{predictions[idx]}</span> 
                    </div>
                    <div>
                        <p>Please label the image again.</p> 
                    </div>
                """))
            add_a_vote("user_votes_1",idx, label)

            first_click = False  # After the first click, change the flag
        else:
            add_a_vote("user_votes_0",idx, label)
            idx += 1
            first_click = True
            show_next_image(images, predictions, idx, img_widget, output_widget)
            
    
    return on_button_click

# Helper function to convert PIL Image to bytes for displaying in ipywidgets
def img_to_bytes(img):
    from io import BytesIO
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    return img_byte_arr.getvalue()

# Function to start the labeling process
def start_labeling(image_paths, predictions):
    if not image_paths:
        print("No images found in the specified directory.")
        return
    
    img_widget = widgets.Image()
    output_widget = widgets.Output()
    display(widgets.HBox([img_widget, output_widget]))  # Display image and output side by side
    
    buttons = [
        widgets.Button(description="Dog"),
        widgets.Button(description="Cat"),
        widgets.Button(description="Wildlife"),
    ]
    
    # Bind the on_button_click event to each button
    on_button_click = show_next_image(image_paths, predictions, 0, img_widget, output_widget)
    for button in buttons:
        button.on_click(lambda b: on_button_click(b.description))
    
    display(widgets.HBox(buttons))

# Example usage
start_labeling(image_paths, predictions)
